In [4]:
# import libraries
import mne, os, pickle, glob
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt


# setting

In [5]:
# directories
main_dir = '/Users/cl5564/Dropbox/Analysis/NTULingAmb/'
os.chdir(main_dir) # change directory
meg_dir = os.path.join(main_dir, '02_meg')  # meg directory
mri_dir = os.path.join(main_dir, '03_mri')  # mri directory
stc_dir = os.path.join(main_dir, '04_stc')  # stc directory

# subject list
subjects = np.arange(7, 25+1, 1)
subjects = np.setdiff1d(subjects, [18, 20]) # exlude subjects: 018, 020
print(subjects)


[ 7  8  9 10 11 12 13 14 15 16 17 19 21 22 23 24 25]


In [3]:
# define variables
tmin = -0.1
tmax = 1.0
baseline = (None, 0)  # tmin~0 , -0.1s~0s   # none = start from tmin.
conditions = ['amb_dom', 'amb_sub', 'unamb']

for s, sid in enumerate(subjects):
    
    # load data
    subj = ('%.3d') %(sid)
    subj_dir = os.path.join(meg_dir, '%s') %(subj)
    epochs_fname = os.path.join(subj_dir, '%s-epo.fif') %(subj)
    epochs = mne.read_epochs(epochs_fname)
    evoked_fname = os.path.join(subj_dir, '%s_ambiguity-ave.fif') %(subj)
    evoked = mne.read_evokeds(evoked_fname)
    info = epochs.info
    
    # define file names for source analysis
    trans_fname = os.path.join(subj_dir, '%s-trans.fif') %(subj)
    bem_fname = os.path.join(mri_dir, '%s', 'bem', '%s-inner_skull-bem-sol.fif') %(subj, subj)
    src_fname = os.path.join(mri_dir, '%s', 'bem', '%s-ico-4-src.fif') %(subj, subj)
    fwd_fname = os.path.join(subj_dir, '%s-fwd.fif') %(subj)
    cov_fname = os.path.join(subj_dir, '%s-cov.fif') %(subj)
    inv_fname = os.path.join(subj_dir, '%s-source-inv.fif') %(subj)
    
    # empty room for denoising
    em_fname = os.path.join(subj_dir, '%sempty.con') %(subj)
    raw_em = mne.io.kit.read_raw_kit(em_fname)
    raw_em.load_data().pick_types(meg=True, stim=False).filter(0, 30, phase= 'zero-double')
    # compute noise covariance
    noise_cov = mne.compute_raw_covariance(raw_em, tmin=0, tmax=None)
    
    # compute the source space
    src = mne.setup_source_space(subject=subj, spacing='ico4', subjects_dir=mri_dir)
    src.save(src_fname, overwrite=True)
    #src = mne.read_source_spaces(src_fname)
    
    # bem model
    conductivity = (0.3,)
    model = mne.make_bem_model(subject=subj, ico=4, conductivity=conductivity, subjects_dir=mri_dir)
    bem = mne.make_bem_solution(model)
    mne.bem.write_bem_solution(bem_fname, bem ,overwrite=True)
    
    # forward model
    fwd = mne.make_forward_solution(info=info, trans=trans_fname, src=src, bem=bem, 
                                    meg=True, eeg=False, mindist=5.0, ignore_ref=True)    
    fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=False, force_fixed=False, use_cps=True)
    #不限制axis旋轉角度=free orientation.
    mne.write_forward_solution(fwd_fname, fwd_fixed, overwrite=True)
    #fwd = mne.read_forward_solution(fwd_fname)

    # inverse operation
    SNR = 3 # 3 for ANOVAs, 2 for regressions
    lambda2 = 1.0 / SNR ** 2
    # free orientation.
    inv = mne.minimum_norm.make_inverse_operator(info, fwd_fixed, noise_cov, loose=1.0, 
                                                 depth=None, fixed=False, verbose=True)
    mne.minimum_norm.write_inverse_operator(inv_fname, inv, overwrite=True)
    
    for c, cond in enumerate(conditions):
        
        # create stc for each condition
        # pick ori can create constraint for signal orientation.
        evk = evoked[c]
        stc = mne.minimum_norm.apply_inverse(evk, inv, lambda2, 'dSPM', verbose=True, pick_ori=None)
        
        # morph to fsaverage template
        morph = mne.compute_source_morph(stc, subject_from=subj, subject_to='fsaverage',
                                         subjects_dir=mri_dir, spacing=4)
        stc_fsavg = morph.apply(stc)
        
        # save stc file
        stc_fname = os.path.join(stc_dir, '%s', '%s_stc_%s') %(cond, subj, cond)
        stc_fsavg.save(stc_fname, overwrite=True)
    

Reading /Users/cl5564/Dropbox/Analysis/NTULingAmb/02_meg/025/025-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...    1000.00 ms
        0 CTF compensation matrices available
Not setting metadata
126 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/cl5564/Dropbox/Analysis/NTULingAmb/02_meg/025/025_ambiguity-ave.fif ...
    Found the data of interest:
        t =    -100.00 ...    1000.00 ms (ambiguous (dominant biasing))
        0 CTF compensation matrices available
        nave = 32 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.1, 0] sec)
    Found the data of interest:
        t =    -100.00 ...    1000.00 ms (ambiguous (subordinate biasing))
        0 CTF compensation matrices available
        nave = 31 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Using up to 900 segments


[Parallel(n_jobs=1)]: Done 157 out of 157 | elapsed:    0.6s finished


Number of samples used : 180000
[done]
Setting up the source space with the following parameters:

SUBJECTS_DIR = /Users/cl5564/Dropbox/Analysis/NTULingAmb/03_mri
Subject      = 025
Surface      = white
Icosahedron subdivision grade 4

>>> 1. Creating the source space...

Doing the icosahedral vertex picking...
Loading /Users/cl5564/Dropbox/Analysis/NTULingAmb/03_mri/025/surf/lh.white...
Mapping lh 025 -> ico (4) ...
    Triangle neighbors and vertex normals...
Loading geometry from /Users/cl5564/Dropbox/Analysis/NTULingAmb/03_mri/025/surf/lh.sphere...
Setting up the triangulation for the decimated surface...
loaded lh.white 2562/163842 selected to source space (ico = 4)

Loading /Users/cl5564/Dropbox/Analysis/NTULingAmb/03_mri/025/surf/rh.white...
Mapping rh 025 -> ico (4) ...
    Triangle neighbors and vertex normals...
Loading geometry from /Users/cl5564/Dropbox/Analysis/NTULingAmb/03_mri/025/surf/rh.sphere...
Setting up the triangulation for the decimated surface...
loaded rh.white

...

    Found   0/  0 points outside using solid angles
    Total 0/157 points inside the surface
Interior check completed in 205.4 ms

Composing the field computation matrix...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Computing MEG at 5124 source locations (free orientations)...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s finished



Finished.
    Cartesian source orientations...
    [done]
Overwriting existing file.
    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written
Computing inverse operator with 157 channels.
    157 out of 157 channels remain after picking
Selected 157 channels
Whitening the forward solution.
Computing rank from covariance with rank=None
    Using tolerance 1.1e-13 (2.2e-16 eps * 157 dim * 3.3  max singular value)
    Estimated rank (mag): 157
    MAG: rank 157 computed from 157 data channels with 0 projectors
    Setting small MAG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of whitened and weighted lead field matrix.
    largest singular value = 6.68741
    scaling factor to adjust the trace = 6.74178e+20 (nchan = 157 nzero = 0)
Overwriting existing file.
Write inverse operator decomposition in /Users/cl5564/Dropbox/Analysis/NTULingAmb/02_meg/025/025-source-inv.